In [ ]:
import glob
import pickle
import numpy as np
from music21 import converter, instrument, note, chord, stream
from keras.utils import np_utils

import os,keras
import tensorflow as tf

In [ ]:
path_to_learning_set = "/content/gdrive/My Drive/MIDI_files"
path_to_note_file = "/content/gdrive/My Drive/Notes"
path_to_model = "/content/gdrive/My Drive/Model.h5"

In [ ]:
learning_rate = 0.002
epochs_number = 150

In [ ]:
try:
  device_name = os.environ['COLAB_TPU_ADDR']
  tpu_adress = 'grpc://' + device_name
  print('TPU found at: {}'.format(tpu_adress))

except KeyError:
  print('No TPU found. Chill. We can fix it.')

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
notes = []

for file in glob.glob(path_to_learning_set + "/test/*.mid"):
    midi = converter.parse(file) # parsing midi to notes and chords

    print('Parsing:', plik)

    notes_to_parse = None

    # for midi with many instruments
    try:
        parts = sorted(instrument.partitionByInstrument(midi).parts, key=lambda p: len(p), reverse=True)
        for r in parts:
            ins = r.getInstrument()
            if 'Percussion' in ins.classes or 'Piano' in ins.classes:
                continue
            else:
                notes_to_parse = p.recurse()
                break
        else:
          notes_to_parse = part[0].recurse()
    except AttributeError: #TODO: i'm not sure what error does partitionByInstrument raise but some day i will be
        notes_to_parse = midi.flat.notes

    for sound in notes_to_parse:
        if isinstance(sound, note.Note): # if it is one note
            notes.append(str(sound.pitch))
        elif isinstance(sound, chord.Chord): # if it is chord, save notes seperated by dot
            notes.append('.'.join(str(n) for n in sound.normalOrder))

# Parse songs from midi to strings
with open(path_to_note_file, 'wb') as path:
    pickle.dump(notes, path)

In [ ]:
sequence_length = 100

# all sounds names
sounds_names = sorted(set(notes))

number_of_sounds = len(sounds_names) # number of possible sounds

# maping
notes_in_int = dict((n, i) for i, n in enumerate(sounds_names))

network_input = []
network_output = []

for i in range(0, len(notes) - sequence_length):
    sequence_input = notes[i:i + sequence_length] # we take this many -> number_of_sounds, notes and put them in sequence_input
    sequence_output = notes[i + sequence_length] # and one sound that is after that sequence
    network_input.append([notes_in_int[n] for n in sequence_input]) # to input we put int representation of sequance
    network_output.append(notes_in_int[sequence_output]) # to output we put int represnation of next note

number_of_patterns = len(network_input) # how many records we have, every record is sequence of sounds + one sound after that sequnce

network_input = np.reshape(network_input, (number_of_patterns, sequence_length, 1))
network_input = network_input / float(number_of_sounds)

network_output = np_utils.to_categorical(network_output)

In [ ]:
tf.config.experimental_connect_to_host(tpu_adress)
resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu_adress)
tf.tpu.experimental.initialize_tpu_system(resolver)
strategy = tf.distribute.experimental.TPUStrategy(resolver)

with strategy.scope():
  # creating optimizer
  opt = tf.compat.v1.train.RMSPropOptimizer(learning_rate)
 
        

  input_layer=tf.keras.layers.Input(shape=(network_input.shape[1], network_input.shape[2]))
  x=tf.keras.layers.LSTM(512, return_sequences=True)(input_layer)
  x=tf.keras.layers.Dropout(0.3)(x)
  x=tf.keras.layers.LSTM(512, return_sequences=True)(x)
  x=tf.keras.layers.Dropout(0.3)(x)
  x=tf.keras.layers.LSTM(512)(x)
  x=tf.keras.layers.Dense(256)(x)
  x=tf.keras.layers.Dropout(0.3)(x)

  # output layer must have as many neurones as possible notes in our program
  output_layer = tf.keras.layers.Dense(number_of_sounds, activation='softmax')(x)
  model_f=tf.keras.models.Model(inputs=input_layer, outputs=output_layer)
  model_f.compile(loss='categorical_crossentropy', optimizer=opt)  # kompilacja modelu

  
# konwersja modelu biblioteki Keras na model TensorFlow przystosowany do pracy z TPU


In [ ]:
# saving better moodels
callback_checkpoint = tf.keras.callbacks.ModelCheckpoint(
    path_to_model,
    monitor='loss',
    verbose=0,
    save_best_only=True,
    mode='min'
)

In [ ]:
# newtork training
model_f.fit(network_input, network_output, epochs=epochs_number, batch_size=1024, callbacks=[callback_checkpoint])